In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
import multiprocessing as mp

mp.cpu_count()

16

In [3]:
iOption0 = 1 # piminus, electron

In [4]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-50M_ccbar_pythia_v0.root","my-50M_phi_ee_embed_helios_00-10_v0.root","my-100M_pi0_gg_embed_helios_new_04-6_v0.root",\
            "my-10M_bbbar_pythia_v7.root","my-100M_piminus_single_hagedorn_04-5_v0.root"]
code_path="../AnaTrain/Run14AuAuLeptonComby/"
file_name=file_names[iOption0]
outfilenames= ['ccbar','phi','electron',',positron','pionminus']#output
outfilepath = "/home/yoren/bnl/PHENIX/ee/ML/"
N_start, N_end = 0, 25e6
N_threads = 1
filename=outfilenames[iOption0]
part_ids=[9,3,3,3]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]

In [5]:
root.gSystem.Load(code_path+"MyEvent_C.so")

0

In [6]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"{code_path}field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [7]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [8]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
soso_event_counter = 0 
for ievent in range(N_real_ev):
    if good_event_counter>10: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
            
    n_hits = myevent.GetNVTXhit()
    jhit=0
    for ihit in range(n_hits):
        if myevent.GetVTXHitEntry(jhit).GetSensor() == 1 :
            myevent.RemoveVTXHitEntry(jhit)
        else: jhit+=1
    
    event_container.correct_beam_offset()
    event_container.Associate_Hits_to_Leptons(5,5,5)
    for itrack in range(myevent.GetNtrack()):
        if myevent.GetEntry(itrack).GetPtPrime()>0.4 and myevent.GetEntry(itrack).GetTrkQuality()>62 and myevent.GetEntry(itrack).GetMcId()>1 and myevent.GetEntry(itrack).GetMcId()<10:
            soso_event_counter+=1
    do_continue = False
    if event_container.GetNBDThit()>0:
        do_continue = True
    if not do_continue:
        continue
    good_event_counter+=event_container.GetNBDThit()
    if True : print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, N VTX hit: {myevent.GetNVTXhit()}, Ntracks: {myevent.GetNtrack()}, NBDTtracks: {event_container.GetNBDThit()}") 

    for ipart in range(event_container.GetNBDThit()):
        print("N hits:",  event_container.GetBDTHitEntry(ipart).GetNBDThit(), ", pt:", round(event_container.GetBDTHitEntry(ipart).GetPt(),3),\
              ", id:",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
        for ibdthit in range(event_container.GetBDTHitEntry(ipart).GetNBDThit()):
            bdthit = event_container.GetBDTHitEntry(ipart).GetBDTHitEntry(ibdthit)
            print(ibdthit, round(bdthit.Getsdphi(0),3),",",round(bdthit.Getsdphi(1),3),",",round(bdthit.Getsdphi(2),3),",",round(bdthit.Getsdphi(3),3),";",\
                  round(bdthit.GetIsTrue(0),3),",",round(bdthit.GetIsTrue(1),3),",",round(bdthit.GetIsTrue(2),3),",",round(bdthit.GetIsTrue(3),3))
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

49722597
Nevent: 0, centrality: 33.0, N VTX hit: 12, Ntracks: 2, NBDTtracks: 1
N hits: 3 , pt: 3.636 , id: electron
0 0.306 , 0.35 , 0.062 , -10.0 ; 1.0 , 1.0 , 1.0 , 0.0
1 0.306 , 0.35 , -0.001 , 0.29 ; 1.0 , 1.0 , 1.0 , 1.0
2 0.306 , 0.301 , -10.0 , 0.29 ; 1.0 , 1.0 , 1.0 , 0.0
4.049326419830322
1.4135183095932007
Nevent: 1, centrality: 3.0, N VTX hit: 28, Ntracks: 1, NBDTtracks: 1
N hits: 1 , pt: 1.9 , id: electron
0 0.015 , -0.027 , 0.517 , -10.0 ; 1.0 , 1.0 , 1.0 , 0.0
1.8962992429733276
0.1310482919216156
Nevent: 12, centrality: 17.0, N VTX hit: 75, Ntracks: 1, NBDTtracks: 1
N hits: 6 , pt: 1.294 , id: electron
0 -0.243 , -3.078 , -4.487 , -10.0 ; 1.0 , 1.0 , 1.0 , 0.0
1 -0.243 , -3.078 , -0.661 , -4.253 ; 1.0 , 1.0 , 1.0 , 1.0
2 -0.243 , 0.539 , -4.837 , -4.253 ; 1.0 , 1.0 , 0.0 , 1.0
3 -0.243 , -3.078 , -2.688 , -3.598 ; 1.0 , 1.0 , 1.0 , 0.0
4 -0.243 , -2.856 , -10.0 , -4.253 ; 1.0 , 1.0 , 1.0 , 0.0
5 -0.243 , -4.269 , -10.0 , -3.598 ; 1.0 , 1.0 , 0.0 , 0.0
3.417142152786255
2

In [9]:
print(good_event_counter,soso_event_counter, ievent)

11 18 67


In [10]:
def write_TTree_to_CVS(N_start_loc, N_end_loc, iter = 0):
    data_array = []
    good_event_counter = 0
    soso_event_counter = 0 
    for ievent in range(N_start_loc, min(N_end_loc,N_real_ev)):
        if (ievent - N_start_loc) % 1e5 == 0: print("N of events:", ievent - N_start_loc, iter)
        if good_event_counter>500000000: break
        myevent.ClearEvent()
        br.GetEntry(ievent)
        event_container.SetEvent(myevent)
        if not myevent.GetNtrack(): continue

        n_hits = myevent.GetNVTXhit()
        jhit=0
        for ihit in range(n_hits):
            if myevent.GetVTXHitEntry(jhit).GetSensor() == 1 :
                myevent.RemoveVTXHitEntry(jhit)
            else: jhit+=1

        event_container.correct_beam_offset()
        event_container.Associate_Hits_to_Leptons(5,5,5)
        soso_event_counter+=1
        do_continue = False
        for ipart in range(myevent.GetNtrack()):
            if event_container.GetNBDThit()>0:
                do_continue = True
        if not do_continue:
            continue
        good_event_counter+=1
        for ipart in range(event_container.GetNBDThit()):
            BDTpart  = event_container.GetBDTHitEntry(ipart)
            for ibdthit in range(BDTpart.GetNBDThit()):

                bdthit = event_container.GetBDTHitEntry(ipart).GetBDTHitEntry(ibdthit)
                data_array.append([BDTpart.GetNBDThit(), BDTpart.GetPt(), BDTpart.GetPhi0(), BDTpart.GetThe0(), BDTpart.GetPhiDC(), BDTpart.GetZDC(), \
                                   BDTpart.GetAlpha(), BDTpart.GetEcore(), BDTpart.GetCentrality(), BDTpart.GetCharge(), BDTpart.GetArm(),\
                                   bdthit.Getsdphi(0), bdthit.Getsdthe(0), bdthit.GetIsTrue(0), bdthit.GetSecondHitPhiR(0), bdthit.GetSecondHitPhiL(0), bdthit.GetSecondHitTheR(0), bdthit.GetSecondHitTheL(0),\
                                   bdthit.Getsdphi(1), bdthit.Getsdthe(1), bdthit.GetIsTrue(1), bdthit.GetSecondHitPhiR(1), bdthit.GetSecondHitPhiL(1), bdthit.GetSecondHitTheR(1), bdthit.GetSecondHitTheL(1),\
                                   bdthit.Getsdphi(2), bdthit.Getsdthe(2), bdthit.GetIsTrue(2), bdthit.GetSecondHitPhiR(2), bdthit.GetSecondHitPhiL(2), bdthit.GetSecondHitTheR(2), bdthit.GetSecondHitTheL(2),\
                                   bdthit.Getsdphi(3), bdthit.Getsdthe(3), bdthit.GetIsTrue(3), bdthit.GetSecondHitPhiR(3), bdthit.GetSecondHitPhiL(3), bdthit.GetSecondHitTheR(3), bdthit.GetSecondHitTheL(3),\
                                    ])


    data = pd.DataFrame(data_array, columns=('NBDThit', 'Pt', 'Phi0', 'The0', 'PhiDC', 'ZDC', 'Alpha', 'Ecore', 'Centrality', 'Charge', 'Arm',\
                                             'sdphi0','sdthe0','IsTrue0','SecondHitPhiR0','SecondHitPhiL0','SecondHitTheR0','SecondHitTheL0',\
                                             'sdphi1','sdthe1','IsTrue1','SecondHitPhiR1','SecondHitPhiL1','SecondHitTheR1','SecondHitTheL1',\
                                             'sdphi2','sdthe2','IsTrue2','SecondHitPhiR2','SecondHitPhiL2','SecondHitTheR2','SecondHitTheL2',\
                                             'sdphi3','sdthe3','IsTrue3','SecondHitPhiR3','SecondHitPhiL3','SecondHitTheR3','SecondHitTheL3'))
    
    float_columns=('Pt', 'Phi0', 'The0', 'PhiDC', 'ZDC', 'Alpha', 'Ecore',\
                   'sdphi0','sdthe0','SecondHitPhiR0','SecondHitPhiL0','SecondHitTheR0','SecondHitTheL0',\
                   'sdphi1','sdthe1','SecondHitPhiR1','SecondHitPhiL1','SecondHitTheR1','SecondHitTheL1',\
                   'sdphi2','sdthe2','SecondHitPhiR2','SecondHitPhiL2','SecondHitTheR2','SecondHitTheL2',\
                   'sdphi3','sdthe3','SecondHitPhiR3','SecondHitPhiL3','SecondHitTheR3','SecondHitTheL3')
    int_columns=('NBDThit','Centrality', 'Charge', 'Arm','IsTrue0','IsTrue1','IsTrue2','IsTrue3')
    
    for float_column in float_columns:
        data[float_column] = data[float_column].astype('float16')
    for int_column in int_columns:
        data[int_column] = data[int_column].astype('int32')
    
    print(data.size, data.shape, good_event_counter, soso_event_counter, ievent+1, ievent - N_start_loc+1, iter)
    filepath = outfilepath+filename+str(iter)
    compression_options = dict(method='zip', archive_name=f'{filename}{iter}.csv')

    data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)

    return [[data.size, data.shape, good_event_counter,soso_event_counter, ievent+1, ievent - N_start_loc+1, iter],data.head(),f'{filepath}.zip']

In [11]:
out = write_TTree_to_CVS(0,10000)

N of events: 0 0
165165 (4235, 39) 1811 3492 10000 10000 0


In [12]:
print(out[0])
out[1]

[165165, (4235, 39), 1811, 3492, 10000, 10000, 0]


,NBDThit,Pt,Phi0,The0,PhiDC,ZDC,Alpha,Ecore,Centrality,Charge,...,SecondHitPhiL2,SecondHitTheR2,SecondHitTheL2,sdphi3,sdthe3,IsTrue3,SecondHitPhiR3,SecondHitPhiL3,SecondHitTheR3,SecondHitTheL3
0,3,3.636719,3.441406,1.622070,3.386719,-11.3125,0.028534,3.748047,33,-1,...,29.703125,4.328125,3.732422,-10.000000,-10.000000,0,-99.00000,46.593750,-99.000000,3.267578
1,3,3.636719,3.441406,1.622070,3.386719,-11.3125,0.028534,3.748047,33,-1,...,29.703125,4.328125,3.732422,0.290527,0.345947,1,50.53125,46.593750,3.416016,3.267578
2,3,3.636719,3.441406,1.622070,3.386719,-11.3125,0.028534,3.748047,33,-1,...,29.703125,-99.000000,3.732422,0.290527,0.345947,0,50.53125,46.593750,3.416016,3.267578
3,1,1.899414,0.405029,1.732422,0.297363,-35.9375,0.054230,2.242188,3,-1,...,25.578125,-9.007812,-7.664062,-10.000000,-10.000000,0,-99.00000,13.515625,-99.000000,8.210938
4,6,1.293945,0.433350,1.809570,0.274170,-53.5625,0.081421,1.275391,17,-1,...,6.062500,0.000000,-1.686523,-10.000000,-10.000000,0,-99.00000,38.187500,-99.000000,-15.187500


In [13]:
def write_TTree_to_CVS_MP(iter):
    delta  = int((N_end - N_start)/N_threads)
    N_start_loc, N_end_loc = N_start + delta*iter, N_start + delta*(iter+1)
    output =  write_TTree_to_CVS(N_start_loc,N_end_loc,iter)
    return output


In [ ]:
Ntr = N_threads

pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(write_TTree_to_CVS_MP,  [i for i in range(Ntr)])
pool.close()

N of events: 0 0
N of events: 100000 0
N of events: 200000 0
N of events: 300000 0
N of events: 400000 0
N of events: 500000 0
N of events: 600000 0
N of events: 700000 0
N of events: 800000 0
N of events: 900000 0
N of events: 1000000 0
N of events: 1100000 0
N of events: 1200000 0
N of events: 1300000 0
N of events: 1400000 0
N of events: 1500000 0
N of events: 1600000 0
N of events: 1700000 0
N of events: 1800000 0
N of events: 1900000 0
N of events: 2000000 0
N of events: 2100000 0
N of events: 2200000 0
N of events: 2300000 0
N of events: 2400000 0
N of events: 2500000 0
N of events: 2600000 0
N of events: 2700000 0
N of events: 2800000 0
N of events: 2900000 0
N of events: 3000000 0
N of events: 3100000 0
N of events: 3200000 0
N of events: 3300000 0
N of events: 3400000 0
N of events: 3500000 0
N of events: 3600000 0
N of events: 3700000 0
N of events: 3800000 0
N of events: 3900000 0
N of events: 4000000 0
N of events: 4100000 0
N of events: 4200000 0
N of events: 4300000 0
N o

In [ ]:
zips = []
for zipname in output_array:
    zips.append(zipname[2])

In [ ]:
print(zips)

['/home/yoren/bnl/PHENIX/ee/ML/phi0.zip']


In [ ]:
import zipfile as z
import os

In [ ]:
with z.ZipFile(zips[0], 'a') as z1:
    for fname in zips[1:]:
        zf = z.ZipFile(fname, 'r')
        for n in zf.namelist():
            z1.writestr(n, zf.open(n).read())
for izip in range(1,len(zips)):    
    os.remove(zips[izip])


In [ ]:
if False:
    data1 = pd.read_csv("/home/yoren/bnl/PHENIX/ee/ML/"+'ccbar0.csv')